In [ ]:
import awkward as ak
import akimbo.pandas
import pandas as pd

In [ ]:
df = pd.read_parquet("out.parquet", dtype_backend="pyarrow")

In [ ]:
df

In [ ]:
df[:0].ak

In [ ]:
nblock = df["blocks"].ak["ip"].ak.num(axis=1)
nblock

In [ ]:
nblock.value_counts().head()

In [ ]:
# replication factor
replication = (df["blocks"]
 .ak["ip"]
 .ak.num(axis=2)
 .ak.flatten()
)
replication

In [ ]:
%timeit df["blocks"].ak["ip"].ak.num(axis=2).ak.flatten()

In [ ]:
%timeit df["blocks"].ak.apply(lambda x: ak.flatten(ak.num(x["ip"], axis=2)))

In [ ]:
replication.value_counts().head()

In [ ]:
# experimental add-on, gives .ak.ip 
import akimbo_ip

In [ ]:
ips = df["blocks"].ak["ip"]
ips.ak.ip.is_global4()  # nothing happens - must be list[4, u8] or bytestring[4]

In [ ]:
def to_fixed(lay, *_, **__):
    """Make bytestrings into fixed-width"""
    if lay.parameters.get("__array__", "") == "bytestring":
        return lay.to_RegularArray()
    
ips2 = ips.ak.transform(to_fixed)

In [ ]:
ips2.ak.array.show(type=True)

In [ ]:
ips2.ak.ip.is_global4().ak.array.show(type=True)

In [ ]:
ips2[:4].ak.ip.to_string4().tolist()

In [ ]:
import ipaddress
%timeit ips2.ak.ip.to_string4()
%timeit [[[str(ipaddress.IPv4Address(_)) for _ in inner or []] for inner in outer] for outer in ips2]